# Anamoly detection using 12 past observations.

## Modules required 

In [31]:
import pandas as pd
import zipfile as zp
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans 
from sklearn.preprocessing import  scale
from json import JSONEncoder
import requests
import  datetime as datetime 
import plotly.express as px
import plotly.graph_objects as go


## commented Code 

In [2]:

# def pivot_reset_tabel( variable_of_intrest ):
#   resetted = variable_of_intrest.reset_index()
#   resetted['time'] = resetted.TimestampLocal.dt.time
#   resetted['Date_only'] = resetted.TimestampLocal.dt.date
#   pivot_resetted = resetted.pivot_table( index  = 'Date_only' , columns='time' , values = variable_of_intrest)
#   return pivot_resetted    

# def month_wise_visualisation( start = '' , end = '' , var = 'AVG in-plane irradiance' , mode = 'MAD' ):
#   global high_ground
#   pivot_resetted = pivot_reset_tabel( var )
#   subset =  pivot_resetted.loc[ pd.to_datetime(start) : pd.to_datetime(end)]
  
#   ##historic_data = pivot_resetted.loc[ pd.to_datetime(start) : pd.to_datetime(end)]
#   ##variable_of_intrest =  pivot_resetted.loc[pd.to_datetime(start):pd.to_datetime(start)]
  
  
#   subset = subset.iloc[:-1,:]

#   high_ground = subset
#   varibles_to_draw = subset.iloc[-1,:]

#   '''
#   Visualisation 
#   '''
#   if( mode == 'MAD' ):
#     f = go.Figure()
#     f.add_trace( go.Scatter( x =  list(subset.columns)  ,  y  = subset.median()    ) )
#     f.add_trace( go.Scatter( x =  list(subset.columns)  ,  y  = (subset.median() + ( 3 * subset.apply(mad,raw=True))) , fill = 'tonexty'    ) )
#     f.add_trace( go.Scatter( x =  list(subset.columns)  ,  y  = (subset.median() - ( 3 * subset.apply(mad,raw=True))) , fill = 'tonexty'    ) )
#     f.add_trace( go.Scatter( x =  list(subset.columns) ,  y  = varibles_to_draw , mode = 'markers'    ) )
#     f.update_layout( title = f'Graph showing Anamolies in {var.upper()} for a period starting from  {start} to {end}' , xaxis_title = 'Time' , yaxis_title = var )
#     f.show()

#   if( mode == 'MEAN' ):
#     f = go.Figure()
#     f.add_trace( go.Scatter( x =  list(subset.columns)  ,  y  = subset.describe().loc['mean']    ) )
#     f.add_trace( go.Scatter( x =  list(subset.columns)  ,  y  = (subset.describe().loc['mean'] + 3 * subset.describe().loc['std']) , fill = 'tonexty'    ) )
#     f.add_trace( go.Scatter( x =  list(subset.columns)  ,  y  = (subset.describe().loc['mean'] - 3 * subset.describe().loc['std']) , fill = 'tonexty'    ) )
#     f.add_trace( go.Scatter( x =  list(subset.columns) ,  y  = varibles_to_draw , mode = 'markers'    ) )
#     f.update_layout( title = f'Graph showing Anamolies in {var.upper()} for a period starting from  {start} to {end}' , xaxis_title = 'Time' , yaxis_title = var )
#     f.show()


## Get Data From API

In [133]:
#Definitions
class DateTimeEncoder(JSONEncoder):
        #Override the default method
        def default(self, obj):
            if isinstance(obj, (datetime.date, datetime.datetime)):
                return obj.isoformat()


def get_data(id,attri,start_date , end_date):
    ''' 
    Edit this if your running a cron Job 
    queryTimeDiff = datetime.timedelta(minutes=90)
    endDate = datetime.datetime.now()
    startDate = datetime.datetime.now() - queryTimeDiff
    '''
    from pandas.tseries.offsets import DateOffset
    ts = pd.to_datetime( start_date  , utc=True)
    x = ts - DateOffset(days=12)
    start_date =   str(x.year) + '-' +  str(x.month) + '-' + str(x.day)  + ' ' +  '00:00:00'  
    #Change the start data and End Data for longer duration 
    query = {'id':id,'attributes':attri,'startDate':start_date,'endDate':end_date}
    query = DateTimeEncoder().encode(query)
    query = eval(query)
    response = requests.get('http://54.206.42.58:8006/api/v2/historicalData/getObjectAttributeHistoricalData', params=query)

    data = response.json()
    exportedData = data["data"]["ObjectData"]
    df = pd.DataFrame(exportedData)
    pivoted = df.pivot( index= 'measurementtimestamp' , columns='attributeserviceid' , values= 'value' )
    #pivoted = pivoted.dropna(how='any',axis=0) 
    return pivoted


mad = lambda x: np.median(np.fabs(x - np.median(x)))

In [134]:
#pivotting data 
strt = '2021-11-13 00:00:00'
end =  '2021-11-15 23:59:00'
df = get_data( 'WP_SF_MVPS1.INV1', 'Active Power', strt ,end )
display(df.head(5))

def pivot_reset_tabel( variable_of_intrest ):
  resetted = variable_of_intrest.reset_index()
  resetted['measurementtimestamp'] =  pd.to_datetime(resetted.measurementtimestamp )
  resetted['time'] = resetted.measurementtimestamp.dt.time
  resetted['Date_only'] = resetted.measurementtimestamp.dt.date
  resetted['Active Power'] = resetted['Active Power'].astype(float)
  pivot_resetted = resetted.pivot_table( index  = 'Date_only' , columns='time' , values = 'Active Power')
  return pivot_resetted  

pivot_resetted = pivot_reset_tabel(df)

attributeserviceid,Active Power
measurementtimestamp,
2021-11-01T00:00:00.000Z,2082
2021-11-01T00:05:00.000Z,2084
2021-11-01T00:10:00.000Z,2079
2021-11-01T00:15:00.000Z,2086
2021-11-01T00:20:00.000Z,2088


In [149]:
60/5

12.0

In [156]:
12 * 24

288

In [177]:
(60 * 24) / 5 

288.0

In [178]:
set(pivot_resetted.columns)

{datetime.time(0, 0),
 datetime.time(0, 5),
 datetime.time(0, 10),
 datetime.time(0, 15),
 datetime.time(0, 20),
 datetime.time(0, 25),
 datetime.time(0, 30),
 datetime.time(0, 35),
 datetime.time(0, 40),
 datetime.time(0, 45),
 datetime.time(0, 50),
 datetime.time(0, 55),
 datetime.time(1, 0),
 datetime.time(1, 5),
 datetime.time(1, 10),
 datetime.time(1, 15),
 datetime.time(1, 20),
 datetime.time(1, 25),
 datetime.time(1, 30),
 datetime.time(1, 35),
 datetime.time(1, 40),
 datetime.time(1, 45),
 datetime.time(1, 50),
 datetime.time(1, 55),
 datetime.time(2, 0),
 datetime.time(2, 4, 59),
 datetime.time(2, 5),
 datetime.time(2, 10),
 datetime.time(2, 15),
 datetime.time(2, 20),
 datetime.time(2, 25),
 datetime.time(2, 25, 1),
 datetime.time(2, 30),
 datetime.time(2, 35),
 datetime.time(2, 40),
 datetime.time(2, 45),
 datetime.time(2, 50),
 datetime.time(2, 55),
 datetime.time(3, 0),
 datetime.time(3, 5),
 datetime.time(3, 10),
 datetime.time(3, 15),
 datetime.time(3, 20),
 datetime.tim

In [174]:
np.unique(pivot_resetted.columns)

array([datetime.time(0, 0), datetime.time(0, 5), datetime.time(0, 10),
       datetime.time(0, 15), datetime.time(0, 20), datetime.time(0, 25),
       datetime.time(0, 30), datetime.time(0, 35), datetime.time(0, 40),
       datetime.time(0, 45), datetime.time(0, 50), datetime.time(0, 55),
       datetime.time(1, 0), datetime.time(1, 5), datetime.time(1, 10),
       datetime.time(1, 15), datetime.time(1, 20), datetime.time(1, 25),
       datetime.time(1, 30), datetime.time(1, 35), datetime.time(1, 40),
       datetime.time(1, 45), datetime.time(1, 50), datetime.time(1, 55),
       datetime.time(2, 0), datetime.time(2, 4, 59), datetime.time(2, 5),
       datetime.time(2, 10), datetime.time(2, 15), datetime.time(2, 20),
       datetime.time(2, 25), datetime.time(2, 25, 1),
       datetime.time(2, 30), datetime.time(2, 35), datetime.time(2, 40),
       datetime.time(2, 45), datetime.time(2, 50), datetime.time(2, 55),
       datetime.time(3, 0), datetime.time(3, 5), datetime.time(3, 10),
  

In [143]:
24 * (60/5) 

288.0

In [144]:
288 * 3

864

In [140]:
df

attributeserviceid,Active Power
measurementtimestamp,
2021-11-01 00:00:00+00:00,2082
2021-11-01 00:05:00+00:00,2084
2021-11-01 00:10:00+00:00,2079
2021-11-01 00:15:00+00:00,2086
2021-11-01 00:20:00+00:00,2088
...,...
2021-11-15 23:35:00+00:00,666
2021-11-15 23:40:00+00:00,542
2021-11-15 23:45:00+00:00,469


In [172]:
#Looping thru dates-computing median MAD

def compute( historical , intrest  ):
    actual = intrest.T
    #display(actual.T)
    median = historical.median()
    #display(median)
    TH = (historical.median() + ( 3 * historical.apply(mad,raw=True)))
    #isplay(TH)
    TL = (historical.median() - ( 3 * historical.apply(mad,raw=True)))
    #display(TL)
    df =  pd.concat( [actual,median ,TH ,TL] , axis = 1 )
    #display(df)
    df.columns =  ['Active Power' , 'Median' , 'TH' , 'TL']  
    df.reset_index( inplace= True)
    #display(df)
    df.drop( columns = ['time'] , inplace= True )
    #display(df)
    return df 
   

start = -1 
ending = 11
df_list = [] 
for each in pd.date_range(start=strt.split( ' ' )[0], end= '2021-11-15'):
    start = start + 1
    ending = ending + 1  
    historical = pivot_resetted.iloc[start:ending,:]
    intrest =    pivot_resetted.loc[ each : each ]
    df =  compute( historical , intrest  )
    print(df.shape)
    df_list.append(compute( historical , intrest  ))

main = pd.concat(df_list)

    

(300, 4)
(300, 4)
(300, 4)


In [145]:
main.groupby(['Active Power']).size()

Active Power
-3.0         3
 0.0       346
 2.0         2
 3.0         1
 4.0         2
          ... 
 2495.0      2
 2496.0      2
 2497.0      3
 2498.0      1
 2499.0      1
Length: 437, dtype: int64

In [161]:
main


,Active Power,Median,TH,TL
0,1711.0,809.0,NaN,NaN
1,1654.0,819.0,2457.0,-819.0
2,1641.0,910.5,2788.5,-967.5
3,1443.0,1106.5,3650.5,-1437.5
4,2327.0,939.5,1877.0,2.0
...,...,...,...,...
295,666.0,778.5,1554.0,3.0
296,542.0,846.5,1709.0,-16.0
297,469.0,840.5,2169.5,-488.5
298,515.0,1016.0,2255.0,-223.0


In [160]:
df.index = pd.to_datetime(df.index)

df.loc[ pd.to_datetime(strt, utc = True) : ]

attributeserviceid,Active Power
measurementtimestamp,
2021-11-13 00:00:00+00:00,1711
2021-11-13 00:05:00+00:00,1654
2021-11-13 00:10:00+00:00,1641
2021-11-13 00:15:00+00:00,1443
2021-11-13 00:20:00+00:00,2327
...,...
2021-11-15 23:35:00+00:00,666
2021-11-15 23:40:00+00:00,542
2021-11-15 23:45:00+00:00,469


In [126]:
print(len(np.unique( main['Active Power'])))
len(np.unique( df.loc[ pd.to_datetime(strt, utc = True) : pd.to_datetime(end, utc = True) ]['Active Power']))


478


437

In [110]:
df.loc[ pd.to_datetime(strt, utc = True) : pd.to_datetime(end, utc = True) ].astype( float ).reset_index()

attributeserviceid,measurementtimestamp,Active Power
0,2021-11-13 00:00:00+00:00,1711.0
1,2021-11-13 00:05:00+00:00,1654.0
2,2021-11-13 00:10:00+00:00,1641.0
3,2021-11-13 00:15:00+00:00,1443.0
4,2021-11-13 00:20:00+00:00,2327.0
...,...,...
854,2021-11-15 23:35:00+00:00,666.0
855,2021-11-15 23:40:00+00:00,542.0
856,2021-11-15 23:45:00+00:00,469.0
857,2021-11-15 23:50:00+00:00,515.0


In [113]:
pd.merge(main,df.loc[ pd.to_datetime(strt, utc = True) : pd.to_datetime(end, utc = True) ].astype( float ).reset_index(),on=['Active Power'])

120411

In [25]:
#Number of zeros 
( pivot_resetted == 0 ).sum( axis=1 )

Date_only
2021-11-01    120
2021-11-02    124
2021-11-03    122
2021-11-04    122
2021-11-05    119
2021-11-06    121
2021-11-07    120
2021-11-08    117
2021-11-09    123
2021-11-10    117
2021-11-11    116
2021-11-12    116
2021-11-13    116
2021-11-14    117
2021-11-15    113
dtype: int64

In [26]:
#Visualisation
main.index.tz_convert('Australia/Sydney')

AttributeError: 'Int64Index' object has no attribute 'tz_convert'

In [183]:
set(pd.date_range(  start='2021-11-13' , end = '2021-11-16' , freq= '5Min' , closed = 'right' ))
df.index = pd.to_datetime(df.index)
set(df.loc[ pd.to_datetime(strt, utc = True) : ].index)

/Users/arvindchandrasekarreddy/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:5278: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  start_slice, end_slice = self.slice_locs(start, end, step=step, kind=kind)


set()

In [184]:
df.loc[ pd.to_datetime(strt, utc = True) : ]

,Active Power,Median,TH,TL


In [71]:
pd.to_datetime(end, utc = True)

Timestamp('2021-11-15 23:59:00+0000', tz='UTC')

In [181]:
df.index = pd.to_datetime(df.index)
df.index = pd.to_datetime(df.index)
main.index =  df.loc[ pd.to_datetime(strt, utc = True) : pd.to_datetime(end, utc = True) ]


/Users/arvindchandrasekarreddy/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:5278: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  start_slice, end_slice = self.slice_locs(start, end, step=step, kind=kind)


ValueError: Index data must be 1-dimensional

In [ ]:

f = go.Figure()
f.add_trace( go.Scatter( x =  list(main.index)  ,  y  = main.Median    ) )
f.add_trace( go.Scatter( x =  list(main.index)   ,  y  = main.TH , fill = 'tonexty'    ) )
f.add_trace( go.Scatter( x =  list(main.index)   ,  y  = main.TL , fill = 'tonexty'    ) )
f.add_trace( go.Scatter( x =  list(main.index)  ,  y  = main['Active Power'] , mode = 'markers'    ) )
#f.update_layout( title = f'Graph showing Anamolies in {var.upper()} for a period starting from  {start} to {end}' , xaxis_title = 'Time')
f.show()

# Foot Notes 

1. .

2. 